In [65]:
# get NSL-KDD dataset
import kagglehub
import pandas as pd

path = kagglehub.dataset_download("hassan06/nslkdd")
print("Path to dataset files:", path)

train_name, test_name = 'KDDTrain+.arff', 'KDDTest+.arff'

# couldn't get arff loading libaries to work, so I'll do it manually
def parse_arff(p):
    with open(p, 'r') as f:
        lines = f.readlines()

        attribute_strings = [l.replace('\n', '').replace('@attribute ', '') for l in lines if l.startswith('@attribute')]

        attributes = {}
        for s in attribute_strings:
            # we only have attribute type real or categorical in this dataset
            att_name = s[0:s.find(' ')].replace('\'', '')
            if s.endswith('real'):
                attributes[att_name] = 'real'
            else:
                attributes[att_name] = eval(s[s.find(' ')+1:])

        data_ind = -1 # data starts at this index
        for i in range(len(lines)):
            if lines[i].find('@data') != -1:
                data_ind = i+1
        
        data = [] 
        positional_attribs = list(attributes.items()) # python 3.7+ guarantees dict order of insertion
        for d in lines[data_ind:]:
            attribs = d.replace('\n', '').split(',')
            row = []
            for i in range(len(attribs)):
                row.append(float(attribs[i]) if positional_attribs[i][1] == 'real' else str(attribs[i]))
            data.append(row)

    return attributes, data

attributes, data = parse_arff(path+'/'+train_name)

print(attributes['service'])

Path to dataset files: /home/joao/.cache/kagglehub/datasets/hassan06/nslkdd/versions/1
{'aol', 'hostnames', 'finger', 'efs', 'gopher', 'http', 'remote_job', 'csnet_ns', 'urh_i', 'auth', 'http_443', 'http_2784', 'tim_i', 'vmnet', 'echo', 'imap4', 'ctf', 'ssh', 'netstat', 'sunrpc', 'ftp_data', 'login', 'private', 'http_8001', 'telnet', 'pop_3', 'red_i', 'exec', 'nnsp', 'ldap', 'printer', 'Z39_50', 'IRC', 'time', 'netbios_dgm', 'rje', 'urp_i', 'eco_i', 'discard', 'courier', 'daytime', 'sql_net', 'link', 'smtp', 'ftp', 'pm_dump', 'uucp', 'X11', 'supdup', 'netbios_ns', 'ntp_u', 'kshell', 'netbios_ssn', 'shell', 'name', 'tftp_u', 'ecr_i', 'bgp', 'nntp', 'uucp_path', 'iso_tsap', 'domain_u', 'mtp', 'systat', 'whois', 'other', 'domain', 'pop_2', 'harvest', 'klogin'}
